## Số lượng ký tự trong Train và Val dataset gốc

In [18]:
### Train Vocab

train_vocab = set()

with open("/bao/Nom-Viet/NomNaOCR/Datasets/Patches/Train.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        chars = parts[-1]
        for ch in chars:
            train_vocab.add(ch)

In [19]:
### Val Vocab

val_vocab = set()

with open("/bao/Nom-Viet/NomNaOCR/Datasets/Patches/Validate.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        chars = parts[-1]
        for ch in chars:
            val_vocab.add(ch)

In [26]:
print("✅ Total vocab - Train:", len(train_vocab))
print("✅ Total vocab - Val:  ", len(val_vocab))


missing_in_train = val_vocab - train_vocab
print("\n🔸 Số ký tự trong Val không có trong Train:", len(missing_in_train))
print("Các ký tự bị missing: \n", list(missing_in_train)[:10])

✅ Total vocab - Train: 7174
✅ Total vocab - Val:   4956

🔸 Số ký tự trong Val không có trong Train: 335
Các ký tự bị missing: 
 ['揇', '猗', '梐', '芀', '迓', '僩', '敄', '𦮎', '𢵯', '\U000f0634']


In [30]:
import csv
from collections import Counter

# --- Cấu hình ---
train_file = "/bao/Nom-Viet/NomNaOCR/Datasets/Patches/Train.txt"
val_file = "/bao/Nom-Viet/NomNaOCR/Datasets/Patches/Validate.txt"
output_csv_file = "vocab_stats.csv"

# --- Khởi tạo counters ---
image_presence_counter = Counter()  # Đếm ký tự có mặt trong bao nhiêu ảnh
total_frequency_counter = Counter()  # Đếm tổng số lần xuất hiện
total_images = 0
total_characters = 0

# --- Hàm xử lý file ---
def process_file(file_path, file_name):
    global total_images, total_characters
    
    print(f"\nĐang xử lý {file_name}: {file_path}...")
    
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            line_count = 0
            char_count = 0
            
            for line in f:
                line = line.strip()
                if not line:
                    continue
                    
                parts = line.split()
                if len(parts) < 2:
                    print(f"  Bỏ qua dòng không hợp lệ: {line}")
                    continue
                
                line_count += 1
                label = parts[-1]  # Lấy phần cuối cùng làm nhãn
                
                # Đếm tổng tần suất
                total_frequency_counter.update(label)
                char_count += len(label)
                
                # Đếm sự hiện diện trong ảnh
                unique_chars_in_line = set(label)
                image_presence_counter.update(unique_chars_in_line)
            
            total_images += line_count
            total_characters += char_count
            
            print(f"  ✓ {file_name}: {line_count} ảnh, {char_count} ký tự")
            
    except FileNotFoundError:
        print(f"  ⚠️ CẢNH BÁO: Không tìm thấy file {file_name}")
    except Exception as e:
        print(f"  ❌ Lỗi khi xử lý {file_name}: {e}")

# --- Xử lý cả Train và Validate ---
print("="*80)
print("BẮT ĐẦU PHÂN TÍCH TỔNG HỢP TRAIN + VALIDATE")
print("="*80)

process_file(train_file, "Train")
process_file(val_file, "Validate")

print("\n" + "="*80)
print("--- THỐNG KÊ TỔNG QUAN (TRAIN + VALIDATE) ---")
print("="*80)
print(f"✅ Tổng số ảnh: {total_images}")
print(f"✅ Tổng số ký tự: {total_characters}")
print(f"✅ Tổng số vocab duy nhất: {len(total_frequency_counter)}")

if total_images == 0 or total_characters == 0:
    print("\n❌ Lỗi: Không tìm thấy dữ liệu hoặc file rỗng.")
else:
    # --- Tính toán thống kê ---
    vocab = sorted(list(total_frequency_counter.keys()))
    stats_list = []
    
    for char in vocab:
        # Số ảnh chứa ký tự này
        img_count = image_presence_counter[char]
        img_ratio = (img_count / total_images) * 100
        
        # Tổng số lần xuất hiện
        freq_count = total_frequency_counter[char]
        freq_ratio = (freq_count / total_characters) * 100
        
        stats_list.append([
            char,
            img_count,
            f"{img_ratio:.4f}%",
            freq_count,
            f"{freq_ratio:.4f}%"
        ])
    
    # Sắp xếp theo tổng tần suất (giảm dần)
    stats_list.sort(key=lambda x: x[3], reverse=True)
    
    # --- Ghi file CSV ---
    header = [
        "ky_tu",
        "so_anh_chua",
        "ty_le_anh",
        "tong_so_lan_xuat_hien",
        "ty_le_tan_suat"
    ]
    
    with open(output_csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(stats_list)
    
    print(f"\n✅ Đã lưu thống kê vào: {output_csv_file}")
    
    # --- Hiển thị Top 20 ---
    print("\n" + "="*80)
    print("TOP 20 KÝ TỰ PHỔ BIẾN NHẤT (theo Tổng số lần xuất hiện)")
    print("="*80)
    print(f"{header[0]:<5} | {header[1]:<12} | {header[2]:<12} | {header[3]:<22} | {header[4]:<15}")
    print("-" * 80)
    
    for row in stats_list[:20]:
        print(f"{row[0]:<5} | {row[1]:<12} | {row[2]:<12} | {row[3]:<22} | {row[4]:<15}")
    
    # --- Phân tích thêm: Vocab chỉ có trong Train hoặc Val ---
    print("\n" + "="*80)
    print("PHÂN TÍCH BỔ SUNG")
    print("="*80)
    
    # Đọc lại để phân tích riêng
    train_vocab = set()
    with open(train_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                chars = parts[-1]
                train_vocab.update(chars)
    
    val_vocab = set()
    with open(val_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                chars = parts[-1]
                val_vocab.update(chars)
    
    only_train = train_vocab - val_vocab
    only_val = val_vocab - train_vocab
    common = train_vocab & val_vocab
    
    print(f"📊 Vocab chỉ có trong Train: {len(only_train)}")
    if only_train and len(only_train) <= 20:
        print(f"   {sorted(only_train)}")
    
    print(f"📊 Vocab chỉ có trong Validate: {len(only_val)}")
    if only_val and len(only_val) <= 20:
        print(f"   {sorted(only_val)}")
    
    print(f"📊 Vocab chung (Train ∩ Validate): {len(common)}")
    print(f"📊 Vocab tổng (Train ∪ Validate): {len(train_vocab | val_vocab)}")

BẮT ĐẦU PHÂN TÍCH TỔNG HỢP TRAIN + VALIDATE

Đang xử lý Train: /bao/Nom-Viet/NomNaOCR/Datasets/Patches/Train.txt...
  ✓ Train: 30654 ảnh, 368441 ký tự

Đang xử lý Validate: /bao/Nom-Viet/NomNaOCR/Datasets/Patches/Validate.txt...
  ✓ Validate: 7664 ảnh, 91106 ký tự

--- THỐNG KÊ TỔNG QUAN (TRAIN + VALIDATE) ---
✅ Tổng số ảnh: 38318
✅ Tổng số ký tự: 459547
✅ Tổng số vocab duy nhất: 7509

✅ Đã lưu thống kê vào: vocab_stats.csv

TOP 20 KÝ TỰ PHỔ BIẾN NHẤT (theo Tổng số lần xuất hiện)
ky_tu | so_anh_chua  | ty_le_anh    | tong_so_lan_xuat_hien  | ty_le_tan_suat 
--------------------------------------------------------------------------------
之     | 6218         | 16.2274%     | 7638                   | 1.6621%        
以     | 4396         | 11.4724%     | 4883                   | 1.0626%        
人     | 3239         | 8.4529%      | 3614                   | 0.7864%        
月     | 3210         | 8.3773%      | 3501                   | 0.7618%        
十     | 3001         | 7.8318%      | 3

## Merge vocab

In [29]:
merge_vocab = val_vocab.union(train_vocab)
print("Tổng số lượng kí tự trong dataset: ", len(merge_vocab))

# Lưu vocab dict
with open("dictionary_7k5.txt", "w", encoding="utf-8") as f:
    for ch in sorted(merge_vocab):
        f.write(ch + "\n")

Tổng số lượng kí tự trong dataset:  7509
